In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import json
import munch
import logging

2023-04-03 13:18:06.798576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Equation(object):
    """Base class for defining PDE related function."""

    def __init__(self, eqn_config):
        self.dim = eqn_config.dim
        self.total_time = eqn_config.total_time
        self.num_time_interval = eqn_config.num_time_interval
        self.delta_t = self.total_time / self.num_time_interval
        self.sqrt_delta_t = np.sqrt(self.delta_t)
        self.y_init = None

    def sample(self, num_sample):
        """Sample forward SDE."""
        raise NotImplementedError

    def f_tf(self, t, x, y, z):
        """Generator function in the PDE."""
        raise NotImplementedError

    def g_tf(self, t, x):
        """Terminal condition of the PDE."""
        raise NotImplementedError


class HJBLQ(Equation):
    """HJB equation in PNAS paper doi.org/10.1073/pnas.1718942115"""
    def __init__(self, eqn_config):
        super(HJBLQ, self).__init__(eqn_config)
        self.x_init = np.zeros(self.dim)
        self.sigma = np.sqrt(2.0)
        self.lambd = 1.0

    def sample(self, num_sample):
        dw_sample = np.random.normal(size=[num_sample, self.dim, self.num_time_interval]) * self.sqrt_delta_t
        x_sample = np.zeros([num_sample, self.dim, self.num_time_interval + 1])
        x_sample[:, :, 0] = np.ones([num_sample, self.dim]) * self.x_init
        for i in range(self.num_time_interval):
            x_sample[:, :, i + 1] = x_sample[:, :, i] + self.sigma * dw_sample[:, :, i]
        return dw_sample, x_sample

    def f_tf(self, t, x, y, z):
        return -self.lambd * tf.reduce_sum(tf.square(z), 1, keepdims=True) / 2

    def g_tf(self, t, x):
        return tf.math.log((1 + tf.reduce_sum(tf.square(x), 1, keepdims=True)) / 2)

In [3]:
class BSDESolver(object):
    """The fully connected neural network model."""
    def __init__(self, config, bsde):
        self.eqn_config = config.eqn_config
        self.net_config = config.net_config
        self.bsde = bsde

        self.model = NonsharedModel(config, bsde)
        self.y_init = self.model.y_init
        lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            self.net_config.lr_boundaries, self.net_config.lr_values)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=1e-8)

    def train(self):
        start_time = time.time()
        training_history = []
        valid_data = self.bsde.sample(self.net_config.valid_size)

        # begin sgd iteration
        for step in range(self.net_config.num_iterations+1):
            if step % self.net_config.logging_frequency == 0:
                loss = self.loss_fn(valid_data, training=False).numpy()
                y_init = self.y_init.numpy()[0]
                elapsed_time = time.time() - start_time
                training_history.append([step, loss, y_init, elapsed_time])
                if self.net_config.verbose:
                    pass
                    logging.info("step: %5u,    loss: %.4e, Y0: %.4e,   elapsed time: %3u" % (step, loss, y_init, elapsed_time))
            self.train_step(self.bsde.sample(self.net_config.batch_size))
        return np.array(training_history)

    def loss_fn(self, inputs, training):
        dw, x = inputs
        y_terminal = self.model(inputs, training)
        delta = y_terminal - self.bsde.g_tf(self.bsde.total_time, x[:, :, -1])
        # use linear approximation outside the clipped range
        loss = tf.reduce_mean(tf.where(tf.abs(delta) < DELTA_CLIP, tf.square(delta),
                                       2 * DELTA_CLIP * tf.abs(delta) - DELTA_CLIP ** 2))

        return loss

    def grad(self, inputs, training):
        with tf.GradientTape(persistent=True) as tape:
            loss = self.loss_fn(inputs, training)
        grad = tape.gradient(loss, self.model.trainable_variables)
        del tape
        return grad

    @tf.function
    def train_step(self, train_data):
        grad = self.grad(train_data, training=True)
        self.optimizer.apply_gradients(zip(grad, self.model.trainable_variables))


class NonsharedModel(tf.keras.Model):
    def __init__(self, config, bsde):
        super(NonsharedModel, self).__init__()
        self.eqn_config = config.eqn_config
        self.net_config = config.net_config
        self.bsde = bsde
        self.y_init = tf.Variable(np.random.uniform(low=self.net_config.y_init_range[0],
                                                    high=self.net_config.y_init_range[1],
                                                    size=[1])
                                  )
        self.z_init = tf.Variable(np.random.uniform(low=-.1, high=.1,
                                                    size=[1, self.eqn_config.dim])
                                  )

        self.subnet = [FeedForwardSubNet(config) for _ in range(self.bsde.num_time_interval-1)]

    def call(self, inputs, training):
        dw, x = inputs
        time_stamp = np.arange(0, self.eqn_config.num_time_interval) * self.bsde.delta_t
        all_one_vec = tf.ones(shape=tf.stack([tf.shape(dw)[0], 1]), dtype=self.net_config.dtype)
        y = all_one_vec * self.y_init
        z = tf.matmul(all_one_vec, self.z_init)

        for t in range(0, self.bsde.num_time_interval-1):
            y = y - self.bsde.delta_t * (
                self.bsde.f_tf(time_stamp[t], x[:, :, t], y, z)
            ) + tf.reduce_sum(z * dw[:, :, t], 1, keepdims=True)
            z = self.subnet[t](x[:, :, t + 1], training) / self.bsde.dim
        # terminal time
        y = y - self.bsde.delta_t * self.bsde.f_tf(time_stamp[-1], x[:, :, -2], y, z) + \
            tf.reduce_sum(z * dw[:, :, -1], 1, keepdims=True)

        return y


class FeedForwardSubNet(tf.keras.Model):
    def __init__(self, config):
        super(FeedForwardSubNet, self).__init__()
        dim = config.eqn_config.dim
        num_hiddens = config.net_config.num_hiddens
        self.bn_layers = [
            tf.keras.layers.BatchNormalization(
                momentum=0.99,
                epsilon=1e-6,
                beta_initializer=tf.random_normal_initializer(0.0, stddev=0.1),
                gamma_initializer=tf.random_uniform_initializer(0.1, 0.5)
            )
            for _ in range(len(num_hiddens) + 2)]
        self.dense_layers = [tf.keras.layers.Dense(num_hiddens[i],
                                                   use_bias=False,
                                                   activation=None)
                             for i in range(len(num_hiddens))]
        # final output should be gradient of size dim
        self.dense_layers.append(tf.keras.layers.Dense(dim, activation=None))

    def call(self, x, training):
        """structure: bn -> (dense -> bn -> relu) * len(num_hiddens) -> dense -> bn"""
        x = self.bn_layers[0](x, training)
        for i in range(len(self.dense_layers) - 1):
            x = self.dense_layers[i](x)
            x = self.bn_layers[i+1](x, training)
            x = tf.nn.relu(x)
        x = self.dense_layers[-1](x)
        x = self.bn_layers[-1](x, training)
        return x

In [4]:
with open('configs/hjb_lq_d100.json') as json_data_file:
    config = json.load(json_data_file)
config = munch.munchify(config)
config.eqn_config

Munch({'_comment': 'HJB equation in PNAS paper doi.org/10.1073/pnas.1718942115', 'eqn_name': 'HJBLQ', 'total_time': 1.0, 'dim': 100, 'num_time_interval': 20})

In [5]:
bsde = HJBLQ(config.eqn_config)
tf.keras.backend.set_floatx(config.net_config.dtype)

In [6]:
DELTA_CLIP = 50.0
bsde_solver = BSDESolver(config, bsde)

In [8]:
dw,dx=bsde.sample(15)

In [11]:
z_0=tf.Variable(np.random.uniform(low=-.1, high=.1,size=[1, config.eqn_config.dim]))

In [19]:
all_one_vec = tf.ones(shape=tf.stack([tf.shape(dw)[0], 1]), dtype=config.net_config.dtype)
all_one_vec

<tf.Tensor: shape=(15, 1), dtype=float64, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])>

In [16]:
z = tf.matmul(all_one_vec, z_0)

In [17]:
z

<tf.Tensor: shape=(15, 100), dtype=float64, numpy=
array([[ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ],
       [ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ],
       [ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ],
       ...,
       [ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ],
       [ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ],
       [ 0.04866279, -0.07039826, -0.04644026, ..., -0.05125745,
        -0.09960487, -0.0743252 ]])>

In [23]:
dw[:,:,2]

array([[-0.01348699, -0.02746936, -0.22185449, ..., -0.01933687,
        -0.08173379, -0.11377345],
       [-0.13954209,  0.33627218, -0.18811262, ..., -0.32140833,
        -0.24545083, -0.02723605],
       [-0.41925457, -0.26709328,  0.10941537, ..., -0.33738343,
        -0.20625324,  0.26551912],
       ...,
       [-0.15071444, -0.09985483, -0.01954619, ...,  0.39601411,
        -0.14645972,  0.07042745],
       [-0.08272666,  0.41288838, -0.19420178, ..., -0.20553366,
        -0.29381084, -0.24761903],
       [-0.19049956,  0.03662557, -0.17463167, ..., -0.27200968,
        -0.09956423,  0.24091191]])

In [20]:
z * dw[:, :, 2]

<tf.Tensor: shape=(15, 100), dtype=float64, numpy=
array([[-0.00065631,  0.0019338 ,  0.01030298, ...,  0.00099116,
         0.00814108,  0.00845623],
       [-0.00679051, -0.02367298,  0.008736  , ...,  0.01647457,
         0.0244481 ,  0.00202432],
       [-0.0204021 ,  0.0188029 , -0.00508128, ...,  0.01729341,
         0.02054383, -0.01973476],
       ...,
       [-0.00733419,  0.00702961,  0.00090773, ..., -0.02029867,
         0.0145881 , -0.00523453],
       [-0.00402571, -0.02906662,  0.00901878, ...,  0.01053513,
         0.02926499,  0.01840433],
       [-0.00927024, -0.00257838,  0.00810994, ...,  0.01394252,
         0.00991708, -0.01790583]])>

In [9]:
training_history = bsde_solver.train()

In [14]:
bsde_solver.bsde.sample(bsde_solver.net_config.valid_size);

In [11]:
bsde_solver.model(bsde_solver.bsde.sample(bsde_solver.net_config.valid_size),)

ValueError: Exception encountered when calling layer 'nonshared_model' (type NonsharedModel).

too many values to unpack (expected 2)

Call arguments received by layer 'nonshared_model' (type NonsharedModel):
  • inputs=tf.Tensor(shape=(100,), dtype=float64)
  • training=None

In [10]:
training_history

array([[0.00000000e+00, 1.96714102e+01, 1.51159449e-01, 5.38725853e-01],
       [1.00000000e+02, 1.10018044e+01, 1.10370134e+00, 1.62500098e+01],
       [2.00000000e+02, 4.81343044e+00, 1.87131667e+00, 1.80582335e+01],
       [3.00000000e+02, 3.02691135e+00, 2.35947189e+00, 1.98888283e+01],
       [4.00000000e+02, 2.46954146e+00, 2.69764328e+00, 2.16959736e+01],
       [5.00000000e+02, 1.90862372e+00, 3.03719509e+00, 2.35058346e+01],
       [6.00000000e+02, 1.38688722e+00, 3.38273458e+00, 2.53116179e+01],
       [7.00000000e+02, 8.65669929e-01, 3.72403470e+00, 2.71200266e+01],
       [8.00000000e+02, 4.59682238e-01, 4.01604072e+00, 2.89222362e+01],
       [9.00000000e+02, 1.85468012e-01, 4.26276027e+00, 3.07416365e+01],
       [1.00000000e+03, 6.47232972e-02, 4.43143586e+00, 3.25491891e+01],
       [1.10000000e+03, 3.02876318e-02, 4.52332243e+00, 3.43766968e+01],
       [1.20000000e+03, 2.27149217e-02, 4.56950927e+00, 3.63222880e+01],
       [1.30000000e+03, 2.16910671e-02, 4.58781897e

In [7]:
np.savetxt('{}_training_history.csv'.format(path_prefix),
               training_history,
               fmt=['%d', '%.5e', '%.5e', '%d'],
               delimiter=",",
               header='step,loss_function,target_value,elapsed_time',
               comments='')

NameError: name 'path_prefix' is not defined

In [8]:
training_history[2]

array([200.        ,   4.71977333,   2.00823993,  18.26610732])